In [2]:
import polars as pl

df_train = pl.read_parquet("../data/output/sft_data/rec_sft_train.parquet").filter(pl.col("task") == "sequence_prediction")
df_val = pl.read_parquet("../data/output/sft_data/rec_sft_val.parquet").filter(pl.col("task") == "sequence_prediction")
df_test = pl.read_parquet("../data/output/sft_data/rec_sft_test.parquet").filter(pl.col("task") == "sequence_prediction")

In [8]:
df_train

conversations,task
list[struct[2]],str
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""根据以下ID序列，请预测下一个应该是什么： <|sid_start|><|sid_61|><|sid_284|><|sid_620|><|sid_768|><|sid_end|>, <|sid_start|><|sid_45|><|sid_357|><|sid_584|><|sid_768|><|sid_end|>, <|sid_start|><|sid_3|><|sid_258|><|sid_553|><|sid_768|><|sid_end|>, <|sid_start|><|sid_60|><|sid_393|><|sid_554|><|sid_768|><|sid_end|>, <|sid_start|><|sid_48|><|sid_322|><|sid_653|><|sid_768|><|sid_end|>, <|sid_start|><|sid_131|><|sid_501|><|sid_673|><|sid_768|><|sid_end|>""}, {""assistant"",""<|sid_start|><|sid_55|><|sid_316|><|sid_712|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""基于用户的历史行为模式（<|sid_start|><|sid_61|><|sid_258|><|sid_611|><|sid_768|><|sid_end|>, <|sid_start|><|sid_182|><|sid_401|><|sid_600|><|sid_768|><|sid_end|>, <|sid_start|><|sid_96|><|sid_334|><|sid_683|><|sid_768|><|sid_end|>, <|sid_start|><|sid_160|><|sid_419|><|sid_590|><|sid_768|><|sid_end|>），请预测下一个最可能出现的ID。""}, {""assistant"",""<|sid_start|><|sid_211|><|sid_275|><|sid_641|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""请补写下一个合理的行为ID，前序列为：<|sid_start|><|sid_28|><|sid_375|><|sid_547|><|sid_768|><|sid_end|>""}, {""assistant"",""<|sid_start|><|sid_207|><|sid_332|><|sid_712|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""历史序列如下：<|sid_start|><|sid_34|><|sid_345|><|sid_640|><|sid_768|><|sid_end|>, <|sid_start|><|sid_34|><|sid_409|><|sid_686|><|sid_768|><|sid_end|>，请补全下一个ID。""}, {""assistant"",""<|sid_start|><|sid_48|><|sid_438|><|sid_544|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""根据以下ID序列，请预测下一个应该是什么： <|sid_start|><|sid_174|><|sid_503|><|sid_693|><|sid_768|><|sid_end|>, <|sid_start|><|sid_67|><|sid_414|><|sid_606|><|sid_768|><|sid_end|>, <|sid_start|><|sid_164|><|sid_412|><|sid_715|><|sid_769|><|sid_end|>, <|sid_start|><|sid_34|><|sid_501|><|sid_674|><|sid_768|><|sid_end|>, <|sid_start|><|sid_34|><|sid_311|><|sid_582|><|sid_768|><|sid_end|>, <|sid_start|><|sid_33|><|sid_313|><|sid_672|><|sid_768|><|sid_end|>, <|sid_start|><|sid_118|><|sid_363|><|sid_600|><|sid_768|><|sid_end|>, <|sid_start|><|sid_160|><|sid_273|><|sid_549|><|sid_768|><|sid_end|>, <|sid_start|><|sid_34|><|sid_311|><|sid_719|><|sid_768|><|sid_end|>, <|sid_start|><|sid_172|><|sid_372|><|sid_683|><|sid_768|><|sid_end|>""}, {""assistant"",""<|sid_start|><|sid_232|><|sid_495|><|sid_737|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
…,…
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""用户已经依次操作了：<|sid_start|><|sid_15|><|sid_364|><|sid_535|><|sid_768|><|sid_end|>, <|sid_start|><|sid_237|><|sid_314|><|sid_569|><|sid_768|><|sid_end|>，请输出可能的下一个ID。""}, {""assistant"",""<|sid_start|><|sid_215|><|sid_490|><|sid_516|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长理解和使用语义ID进行商品推荐和用户偏好分析。""}, {""user"",""用户浏览序列：<|sid_start|><|sid_244|><|sid_321|><|sid_689|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_387|><|sid_514|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_375|><|sid_573|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_447|><|sid_675|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_282|><|sid_562|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_381|><|sid_562|><|sid_768|><|sid_end|>, <|sid_start|><|sid_28|><|sid_411|><|sid_525|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_409|><|sid_641|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_282|><|sid_712|><|sid_768|><|sid_end|>, <|sid_start|><|sid_244|><|sid_483|><|sid_689|><|sid_768|><|sid_end|>，请预测下一个商品ID。""}, {""assistant"",""<|sid_start|><|sid_244|><|sid_403|><|sid_713|><|sid_768|><|sid_end|>""}]","""sequence_prediction"""
"[{""system"",""你是一个专业的商品推荐助手，擅长

In [10]:
import re

def extract_hist_and_target(convs):
    """
    convs: List of dicts with keys:
      - 'role': one of ['system', 'user', 'assistant']
      - 'content': message content
    Returns: (history_seq:str, target_item:str)
    """
    # 1. 找user消息
    user_msg = None
    user_msg = None
    target_item = None
    for m in convs:
        if user_msg is None and m.get('role') == 'user':
            user_msg = m.get('content')
        if target_item is None and m.get('role') == 'assistant':
            assist_content = m.get('content')
            mobj = re.search(r'<\|sid_start\|>.*?<\|sid_end\|>', assist_content)
            target_item = mobj.group(0) if mobj else assist_content
        if (user_msg is not None) and (target_item is not None):
            break

    if user_msg is not None:
        history_items = re.findall(r'<\|sid_start\|>.*?<\|sid_end\|>', user_msg)
        history_seq = ', '.join(history_items) if history_items else None
    else:
        history_seq = None
        print("No user message found")
    return (history_seq, target_item)

def extract_df(df):
    # 使用polars表达式直接处理
    extracted = df.select([
        pl.col("conversations").map_elements(lambda convs: extract_hist_and_target(convs)[0], return_dtype=pl.Utf8).alias("history_seq"),
        pl.col("conversations").map_elements(lambda convs: extract_hist_and_target(convs)[1], return_dtype=pl.Utf8).alias("target_item"),
    ])
    # 在函数内加上字符串分割
    extracted = extracted.with_columns(
        pl.col("history_seq").str.split(",").alias("history_seq")
    )
    return extracted

df_extracted_train = extract_df(df_train)
df_extracted_val   = extract_df(df_val)
df_extracted_test  = extract_df(df_test)

In [11]:
df_extracted_train = df_extracted_train.with_columns(pl.lit("train").alias("split"))
df_extracted_val = df_extracted_val.with_columns(pl.lit("val").alias("split"))
df_extracted_test = df_extracted_test.with_columns(pl.lit("test").alias("split"))

df_extracted = pl.concat([df_extracted_train, df_extracted_val, df_extracted_test])

In [12]:
# 首先找到所有unique的item
# 注意history_seq是list[str], target_item是str
import itertools

# 展开history_seq and target_item为一维
all_history_items = list(itertools.chain.from_iterable(df_extracted['history_seq']))
all_target_items = df_extracted['target_item'].to_list()

all_items_set = set(all_history_items) | set(all_target_items)
all_items = sorted(all_items_set)  # 可以保持排序一致性

item2id = {item: i for i, item in enumerate(all_items)}

# 替换history_seq和target_item为数字编码
df_encoded = df_extracted.with_columns([
    pl.col("history_seq").map_elements(lambda seq: [item2id[x] for x in seq], return_dtype=pl.List(pl.Int32)).alias("history_seq"),
    pl.col("target_item").map_elements(lambda x: item2id[x], return_dtype=pl.Int32).alias("target_item")
])

id2item = {idx: item for item, idx in item2id.items()}

df_encoded

history_seq,target_item,split
list[i32],i32,str
"[42007, 17062, … 3347]",41156,"""train"""
"[41983, 8736, … 5951]",33602,"""train"""
[37153],33310,"""train"""
"[38386, 15583]",40512,"""train"""
"[30845, 19681, … 7300]",35261,"""train"""
…,…,…
"[25476, 9377]",32395,"""test"""
"[34174, 14624, … 17959]",34104,"""test"""
"[29438, 8128, 1575]",34629,"""test"""


In [13]:
import pickle

# 保存 df_encoded 到 parquet 格式（兼容性强）
df_encoded.write_parquet("../data_recbole/df_encoded.parquet")

# 保存 id2item 和 item2id 到 pickle 文件
with open("../data_recbole/id2item.pkl", "wb") as f:
    pickle.dump(id2item, f)

with open("../data_recbole/item2id.pkl", "wb") as f:
    pickle.dump(item2id, f)
